# Machine Translation using Sequence to Sequence

In [1]:
# Importing required libraries

In [3]:
import os
import re
import string
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

In [4]:
# readin dataset

In [7]:
df = pd.read_table("./data/cmn.txt", names=['eng', 'chin', 'info'])
df.head()

,eng,chin,info
0,Hi.,嗨。,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,Hi.,你好。,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Run.,你用跑的。,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
3,Wait!,等等！,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
4,Wait!,等一下！,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


In [8]:
# getting shape of dataset
df.shape

(24026, 3)

In [9]:
# we do not need info so only taking eng and chin features

In [10]:
df = df[["eng", "chin"]]

In [11]:
df.head()

,eng,chin
0,Hi.,嗨。
1,Hi.,你好。
2,Run.,你用跑的。
3,Wait!,等等！
4,Wait!,等一下！


# Data Preprocessing

We are folowing some of the give preprocessing steps:
* Removing punctuations like . , ! $( ) * % @
* Removing URLs
* Removing Stop words
* Lower casing
* Tokenization
* Stemming
* Lemmatization

#### Removing Punctuation

In [24]:
# getting punctuation form string
set_of_punctuation = set(string.punctuation)
print(set_of_punctuation)

{'[', '%', '}', '.', '"', '+', '~', ':', '/', '&', '-', '>', ')', '=', '|', '*', '$', '<', '#', '_', '(', ']', '`', '?', ',', '!', '^', '@', '\\', "'", '{', ';'}


In [36]:
# define a function to remove punctuation form whole dataset
def remove_punctuation(text):
    punctuation_free_text = ''.join([char for char in text if char not in set_of_punctuation])
    return punctuation_free_text

In [37]:
#example
temp = df["eng"].apply(lambda x: remove_punctuation(x))
temp.head()

0      Hi
1      Hi
2     Run
3    Wait
4    Wait
Name: eng, dtype: object

#### Removing Uneven Spaces

In [46]:
# define a function to remove uneven spaces form text
def remove_uneven_spaces(text):
    uneven_space_free = re.sub(r"\s+", ' ', text.strip())
    return uneven_space_free

In [48]:
# example
text = "  This  is   an example    text   with uneven   spaces.  "
result = df["eng"].apply(lambda x: remove_uneven_spaces(text))
result.head()

0    This is an example text with uneven spaces.
1    This is an example text with uneven spaces.
2    This is an example text with uneven spaces.
3    This is an example text with uneven spaces.
4    This is an example text with uneven spaces.
Name: eng, dtype: object

#### Lower casing

In [49]:
# define a functio to convert text into lower case
def convert_lowercase(text):
    return text.lower()

In [ ]:
# example
text = "My Name "

#### Tokenization

# Model Building

Here, we are going to build encoder and decoder models seperatly and ensembel them to form seq2seq model


#### Building Encoder

In [13]:
class Encoder(nn.Module):
    def __init__(self):
        pass
    
    def forward(self, x):
        pass

#### Building Decoder

In [14]:
class Decoder(nn.Module):
    def __init__(self):
        pass
    
    def forward(self, x):
        pass

#### Building Seq2Seq model

In [15]:
class Seq2SeqNet(nn.Module):
    def __init__(self):
        pass
    
    def forward(self, x):
        pass

# Training Model